In [1]:
import os
import warnings
import random
from os.path import join
import torch
import numpy as np
import pandas as pd
from scipy import sparse

In [2]:
torch.__version__

'1.10.0'

In [3]:
from utils import eval_sequential
import warnings
import random
from os.path import join
import torch
import numpy as np
import json

Using backend: pytorch


In [4]:
warnings.filterwarnings('ignore')

# set seed
def seed_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

seed = 1
seed_everything(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
top_k = 20

In [5]:
f = open('data/unique_song_spotify.txt', 'r')
data = f.read()
f.close()

data = data.split("\n")
data = [i.split("\t") for i in data[:-1]]

track2id = dict()
id2track = dict()

for i in data:
    track2id[i[2]] = int(i[-1])
    id2track[int(i[-1])] = i[2]

In [6]:
import json

with open('data/challenge_set.json', 'r') as f:
    data = json.load(f)

In [7]:
playlists = []

for playlist in data["playlists"]:
    if len(playlist['tracks'])>2:
        pl = []
        for track in playlist['tracks']:
            pl.append(track2id[track['track_uri']])
            
        playlists.append(pl)
        

num_users = len(playlists)
num_items = len(track2id)

print(f"# of users: {num_users},  # of items: {num_items}")

music_data = dict()
for idx, i in enumerate(playlists):
    music_data[idx] = i
    
    
user_train_music = dict()
user_valid_music = dict()
user_test_music = dict()

for u in music_data:
    user_valid_music[u] = [music_data[u][-2]]
    user_test_music[u] = [music_data[u][-1]]
    user_train_music[u] = music_data[u][:-2]

# of users: 8000,  # of items: 66243


In [8]:
top_k = 100

In [10]:
"""
TransRec

"""
from models.TransRec_sequential import TransRec_sequential

seed_everything(seed)
TransRec = TransRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, emb_dim=200, maxlen=1,
                                num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 500)
TransRec.fit()
TransRec_ndcg, TransRec_recall = eval_sequential(TransRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k = 100, mode='test')
print(f"\n[TransRec]\t Test_Recall@{top_k} = {TransRec_recall:.4f} Test_NDCG@{top_k} = {TransRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in TransRec.parameters() if p.requires_grad))
print("======================================")

  6%|██████▊                                                                                                     | 4/63 [00:02<00:31,  1.85it/s]


KeyboardInterrupt: 

In [11]:
"""
SASRec
"""
from models.SASRec_sequential import SASRec_sequential

seed_everything(seed)
SASRec = SASRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, hidden_dim=50, maxlen=20, num_blocks=1, num_heads=1,
                            num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 100)
SASRec.fit()
SASRec_ndcg, SASRec_recall = eval_sequential(SASRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k, mode='test')
print(f"\n[SASRec]\t Test_Recall@{top_k} = {SASRec_recall:.4f} Test_NDCG@{top_k} = {SASRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in SASRec.parameters() if p.requires_grad))

valid: 100%|██████████| 1000/1000 [00:14<00:00, 70.50it/s]


epoch 1 train_loss = 5.4659 valid_recall@100 = 0.0000 valid_ndcg@100 = 0.0000


valid: 100%|██████████| 1000/1000 [00:14<00:00, 70.32it/s]


epoch 2 train_loss = 4.8709 valid_recall@100 = 0.0010 valid_ndcg@100 = 0.0002


valid: 100%|██████████| 1000/1000 [00:14<00:00, 70.19it/s]


...

epoch 10 train_loss = 1.4153 valid_recall@100 = 0.0020 valid_ndcg@100 = 0.0005


valid: 100%|██████████| 1000/1000 [00:14<00:00, 70.26it/s]


Early stop at epoch:11


test: 100%|██████████| 8000/8000 [01:54<00:00, 70.13it/s]


[SASRec]	 Test_Recall@500 = 0.0104 Test_NDCG@500 = 0.0014
parmas : 3328800


In [12]:
"""
BERT4Rec
"""
from models.BERTRec_sequential import BERTRec_sequential

seed_everything(seed)
BERTRec = BERTRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, hidden=50*5, maxlen=50, n_layers=2, heads=5, mask_prob=0.1,
                                num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 100)
BERTRec.fit()
BERTRec_ndcg, BERTRec_recall = eval_sequential(BERTRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k = 100, mode='test')
print(f"\n[BERTRec]\t Test_Recall@{top_k} = {BERTRec_recall:.4f} Test_NDCG@{top_k} = {BERTRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in BERTRec.parameters() if p.requires_grad))

valid: 100%|██████████| 1000/1000 [00:25<00:00, 39.56it/s]


epoch 1 train_loss = 10.7780 valid_recall@100 = 0.0340 valid_ndcg@100 = 0.0080


valid: 100%|██████████| 1000/1000 [00:25<00:00, 39.46it/s]


epoch 2 train_loss = 10.7506 valid_recall@100 = 0.0370 valid_ndcg@100 = 0.0098


valid: 100%|██████████| 1000/1000 [00:25<00:00, 39.87it/s]


...

epoch 16 train_loss = 9.8465 valid_recall@100 = 0.0580 valid_ndcg@100 = 0.0130


valid: 100%|██████████| 1000/1000 [00:25<00:00, 39.19it/s]


Early stop at epoch:17


test: 100%|██████████| 8000/8000 [03:23<00:00, 39.23it/s]


[BERTRec]	 Test_Recall@500 = 0.1573 Test_NDCG@500 = 0.0245
parmas : 34707494


In [14]:
train_rows = []
for user in user_train_music:
    for song in user_train_music[user]:
        train_rows.append((user,song,1))
train = pd.DataFrame(train_rows, columns = ['user_id', 'item_id', 'rating'])

In [15]:
valid_rows = []
for user in user_valid_music:
    for song in user_valid_music[user]:
        valid_rows.append((user,song,1))
valid = pd.DataFrame(valid_rows, columns = ['user_id', 'item_id', 'rating'])

In [16]:
test_rows = []
for user in user_test_music:
    for song in user_test_music[user]:
        test_rows.append((user,song,1))
test = pd.DataFrame(test_rows, columns = ['user_id', 'item_id', 'rating'])

In [17]:
train = train.to_numpy()
valid = valid.to_numpy()
test = test.to_numpy()

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in train:
    matrix[u, i] = r
train = sparse.csr_matrix(matrix)

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in valid:
    matrix[u, i] = r
valid = sparse.csr_matrix(matrix)

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in test:
    matrix[u, i] = r
test = sparse.csr_matrix(matrix)

# train = train.toarray()
# valid = valid.toarray()
# test = test.toarray()

In [15]:
top_k = 500

In [11]:
"""
EASE
"""
from models.EASE_implicit import EASE_implicit

seed_everything(seed)
ease = EASE_implicit(train=train, valid=valid, reg_lambda=100)
ease.fit()
train = train+valid
ease_prec, ease_recall, ease_ndcg = eval_implicit(ease, train, test, top_k)
print("EASE Prec@50: %f, Recall@50: %f, NDCG@50: %f"%(ease_prec, ease_recall, ease_ndcg))

NameError: name 'train' is not defined